<a href="https://colab.research.google.com/github/MariaGodoi/cn1t2/blob/main/Projeto_3_Final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp
import numpy as np
import time

# Mapeando funções e constantes comumente usadas
func_map = {
    "sin": sp.sin,
    "cos": sp.cos,
    "tan": sp.tan,
    "exp": sp.exp,
    "log": sp.log,
    "ln": sp.log,  # ln é mapeado para log no SymPy
    "pi": sp.pi
}

# Função para solicitar a função a ser integrada
def ask_for_function():
    function = input("Digite a função a ser integrada (use x, y, z, ... para as variáveis):\n")
    return function

# Função para detectar variáveis a partir da string da função
def detect_variables(function_str):
    expr = sp.sympify(function_str)
    variables = list(expr.free_symbols)
    return sorted(variables, key=lambda x: str(x))

# Função para solicitar limites de integração
def ask_for_limits(variables):
    limits = []
    context = {**func_map}
    for var in variables:
        lower_limit = eval(input(f"Digite o limite inferior de {var} (use 'pi' se necessário): "), context)
        upper_limit = eval(input(f"Digite o limite superior de {var} (use 'pi' se necessário): "), context)
        limits.append((lower_limit, upper_limit))
    return limits

# Função de integração de Monte Carlo
def monte_carlo_integration(func, bounds, num_points):
    dimensions = len(bounds) # Calcula a dimensão do problema (número de variáveis)
    volume = np.prod([b[1] - b[0] for b in bounds])

    random_points = np.random.rand(num_points, dimensions) # Gera pontos aleatórios uniformemente distribuídos entre 0 e 1
    for i in range(dimensions):
        random_points[:, i] = random_points[:, i] * (bounds[i][1] - bounds[i][0]) + bounds[i][0] # Ajusta os pontos aleatórios para ficarem dentro dos limites de integração especificados

    y_random = np.apply_along_axis(lambda x: func(*x), 1, random_points) # Avalia a função nos pontos aleatórios ajustados
    mean_value = np.mean(y_random) # Calcula o valor médio dos resultados da função
    integral = volume * mean_value

    return integral

# Métodos de integração numérica para funções univariadas
def trapezoidal_rule(func, a, b, n):
    x = np.linspace(a, b, n+1)
    y = func(x)
    h = (b - a) / n
    integral = (h / 2) * (y[0] + 2 * np.sum(y[1:n]) + y[n])
    return integral

def simpson_1_3_rule(func, a, b, n):
    if n % 2 == 1:
        n += 1
    x = np.linspace(a, b, n+1)
    y = func(x)
    h = (b - a) / n
    integral = (h / 3) * (y[0] + 4 * np.sum(y[1:n:2]) + 2 * np.sum(y[2:n-1:2]) + y[n])
    return integral

def simpson_3_8_rule(func, a, b, n):
    if n % 3 != 0:
        n += 3 - (n % 3)
    x = np.linspace(a, b, n+1)
    y = func(x)
    h = (b - a) / n
    integral = (3 * h / 8) * (y[0] + 3 * np.sum(y[1:n:3]) + 3 * np.sum(y[2:n:3]) + 2 * np.sum(y[3:n-1:3]) + y[n])
    return integral

# Função principal
def main():
    function_str = ask_for_function()
    variables = detect_variables(function_str)

    print(f"Variáveis detectadas: {', '.join(map(str, variables))}")

    limits = ask_for_limits(variables)

    context = {**func_map}
    context.update({str(symbol): symbol for symbol in variables})
    function_sympy = eval(function_str, context)

    user_function = sp.lambdify(variables, function_sympy, modules=["numpy"])

    print()
    num_points = int(input("Digite o número de pontos de amostragem para Monte Carlo: "))
    start_time = time.time()
    try:
        result_monte_carlo = monte_carlo_integration(user_function, limits, num_points)
    except Exception as e:
        print(f"Erro na avaliação da função com Monte Carlo: {e}")
        return
    end_time = time.time()
    time_monte_carlo = end_time - start_time

    if len(variables) == 1:
        x = variables[0]
        a, b = limits[0]

        print()
        n = int(input("Digite o número de subdivisões para os métodos numéricos: "))

        start_time = time.time()
        try:
            result_trapezoidal = trapezoidal_rule(user_function, a, b, n)
        except Exception as e:
            print(f"Erro na avaliação da função com a regra do trapézio: {e}")
            return
        end_time = time.time()
        time_trapezoidal = end_time - start_time

        start_time = time.time()
        try:
            result_simpson_1_3 = simpson_1_3_rule(user_function, a, b, n)
        except Exception as e:
            print(f"Erro na avaliação da função com a regra de 1/3 de Simpson: {e}")
            return
        end_time = time.time()
        time_simpson_1_3 = end_time - start_time

        start_time = time.time()
        try:
            result_simpson_3_8 = simpson_3_8_rule(user_function, a, b, n)
        except Exception as e:
            print(f"Erro na avaliação da função com a regra de 3/8 de Simpson: {e}")
            return
        end_time = time.time()
        time_simpson_3_8 = end_time - start_time

        print("*Resultados da Integração*")
        print()
        print(f"Método de Monte Carlo:")
        print(f"  Integral aproximada: {result_monte_carlo}")
        print(f"  Tempo de execução: {time_monte_carlo:.6f} segundos")
        print()
        print(f"Regra do Trapézio:")
        print(f"  Integral aproximada: {result_trapezoidal}")
        print(f"  Tempo de execução: {time_trapezoidal:.6f} segundos")
        print()
        print(f"Regra de 1/3 de Simpson:")
        print(f"  Integral aproximada: {result_simpson_1_3}")
        print(f"  Tempo de execução: {time_simpson_1_3:.6f} segundos")
        print()
        print(f"Regra de 3/8 de Simpson:")
        print(f"  Integral aproximada: {result_simpson_3_8}")
        print(f"  Tempo de execução: {time_simpson_3_8:.6f} segundos")

    else:
        start_time = time.time()
        try:
            integral_sympy = sp.integrate(function_sympy, *[(variables[i], limits[i][0], limits[i][1]) for i in range(len(variables))]) # Para múltiplas variáveis, calcula pelo sympy
            result_sympy = integral_sympy.evalf()  # Avalia a integral para um número de ponto flutuante
        except Exception as e:
            print(f"Erro na avaliação da função com SymPy: {e}")
            return
        end_time = time.time()
        time_sympy = end_time - start_time

        print()
        print(f"Método de Monte Carlo:")
        print(f"  Integral aproximada: {result_monte_carlo}")
        print(f"  Tempo de execução: {time_monte_carlo:.6f} segundos")
        print()
        print(f"Método de Integração Simbólica (SymPy):")
        print(f"  Integral exata: {result_sympy}")
        print(f"  Tempo de execução: {time_sympy:.6f} segundos")

if __name__ == "__main__":
    main()

Digite a função a ser integrada (use x, y, z, ... para as variáveis):
sin(x)*cos(y)
Variáveis detectadas: x, y
Digite o limite inferior de x (use 'pi' se necessário): 0
Digite o limite superior de x (use 'pi' se necessário): pi/2
Digite o limite inferior de y (use 'pi' se necessário): 0
Digite o limite superior de y (use 'pi' se necessário): pi/2

Digite o número de pontos de amostragem para Monte Carlo: 1000

Método de Monte Carlo:
  Integral aproximada: 0.100535026149155*pi**2
  Tempo de execução: 0.866957 segundos

Método de Integração Simbólica (SymPy):
  Integral exata: 1.00000000000000
  Tempo de execução: 0.032719 segundos
